In [93]:
import os,csv,codecs,sys
import mysql.connector

#list company
#comlist=[]
#with open('E:/daily/listcompany.csv') as csvfile:
#    reader=csv.reader(csvfile)
#    for row in reader:
#        comlist.append(row[0])
        
#comlistOTC=[]
#with open('E:/daily/listcompanyOTC.csv') as csvfile:
#    reader=csv.reader(csvfile)
#    for row in reader:
#        comlistOTC.append(row[0])
#print len(comlist),len(comlistOTC)

#s3474='E:/daily/raw/20150525/3474.csv'
#s8086='E:/daily/raw/20150525/8086.csv'
#s2348='E:/daily/raw/20150526/2348.csv'
#s3067='E:/daily/raw/20150526/3067.csv'
#stockid=os.path.basename(s8086).split('.')[0]
code=''
rawdatapath='E:/daily/raw/'
#folders=os.listdir(rawdatapath)
folders=['20150522','20150525']
for folder in folders:
    datefolder=os.path.join(rawdatapath,folder)
    datefiles=os.listdir(datefolder)
    for stockfile in datefiles:
        stockid=stockfile
        TimeId=folder
        dailyfilepath=os.path.join(datefolder,stockfile)
        records=[]
        try:            
            with open(dailyfilepath,'r') as dailyfile:
                #print dailyfilepath
                #if stockid in comlist:
                #    code='big5'    
                #elif stockid in comlistOTC:
                #    code='utf-8'
                reader =csv.reader(dailyfile)
                headline=reader.next()
                #if  headline[0].decode(code) not in [u'券商買賣證券成交價量資訊',u'券商買賣股票成交價量資訊']:
                #    print 'no data or wrong file: '+dailyfilepath
                #    continue
                reader.next()
                reader.next()    
                for row in reader:
                    record1={'id':'','broker':'','price':'','buy':'','sell':''}
                    record2={'id':'','broker':'','price':'','buy':'','sell':''}
                    #data=row.decode(code).split(',')
                    record1['id']=row[0]
                    record1['broker']=row[1][:4]
                    record1['price']=float(row[2])
                    record1['buy']=int(row[3].replace(',',''))
                    record1['sell']=int(row[4].replace(',',''))
                    records.append(record1)
                    #print record1               
                    if len(row)>5 and len(row[6])>0:
                        record2['id']=row[6]
                        record2['broker']=row[7][:4]
                        record2['price']=float(row[8])
                        record2['buy']=int(row[9].replace(',',''))
                        record2['sell']=int(row[10].replace(',',''))
                        records.append(record2)
                        #print record2     

        
            cnx = mysql.connector.connect(user='threela', password='123456',
                                          host='10.120.31.4',
                                          database='threela')
            cursor = cnx.cursor()
            add_daily= ("INSERT INTO `threela`.`daily_tmp` "
                           "(StockId,BrokerId,TimeId,Buy,Sell,BuyPrice,SellPrice) "
                           "VALUES (%s,%s,%s,%s,%s,%s,%s)")
            data=[]
            for record in records:
                brokerid=record['broker']
                
                Buy=int(record['buy']/1000)
                Sell=int(record['sell']/1000)
                buyprice=0
                sellprice=0
                if record['buy'] >0:
                        buyprice=record['price']
                if record['sell'] >0:
                        sellprice=record['price']
                data_daily=(stockid,brokerid,TimeId,Buy,Sell,buyprice,sellprice)
                data.append(data_daily)
        
            cursor.executemany(add_daily, data)
            cnx.commit()
        except mysql.connector.Error as err:
            print ("Something went wrong: {}".format(err))
        except:
            print 'Error: '+dailyfilepath
            print "Unexpected error:", sys.exc_info()[0]
            continue
        finally:
            cursor.close()
            cnx.close()

    



        


Error: E:/daily/raw/20150525\3126.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\3322.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\3562.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\3658.csv
Unexpected error: <type 'exceptions.ValueError'>
Error: E:/daily/raw/20150525\4304.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\4416.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\4419.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\5011.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\5209.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\5505.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/daily/raw/20150525\5506.csv
Unexpected error: <type 'exceptions.StopIteration'>
Error: E:/dai

In [ ]:
'''        
content=''
for r in records:
    content+=r['id']+','+r['broker']+','+str(r['price'])+','+str(r['buy'])+','+str(r['sell'])+'\n'
with open('E:/daily/tmp.txt','w') as tmp:    
    tmp.write(content)
    
SumRecords={}
for record in records:
    if SumRecords.has_key(record['broker']):
        print record['broker'] + ' 已存在'
    else:
        buyprice=0
        selprice=0
        buy=int(record['buy']/1000)
        sell=int(record['sell']/1000)
        if record['buy'] >0:
            buyprice=record['price']
        if record['sell'] >0:
            sellprice=record['price']
        SumRecords[record['broker']]={'buy':record['buy'],'sell':record['sell'],'buyprice':buyprice,'sellprice':sellprice}
print SumRecords
'''
        

In [92]:
s3474='E:/daily/raw/20150525/3474.csv'
s8086='E:/daily/raw/20150525/8086.csv'
s2348='E:/daily/raw/20150526/2348.csv'
s3067='E:/daily/raw/20150526/3067.csv'
with open(s8086,'r') as dailyfile:
#print dailyfilepath
    if stockid in comlist:
        code='big5'    
    elif stockid in comlistOTC:
        code='utf-8'
    reader =csv.reader(dailyfile)
    headline=reader.next()
    print headline[0].decode('utf-8')
    if  headline[0].decode('utf-8') not in [u'券商買賣證券成交價量資訊',u'券商買賣股票成交價量資訊']:
        print 'no data or wrong file: '

券商買賣證券成交價量資訊
